In [1]:
!pip install sentence-transformers faiss-cpu

!pip install matplotlib scikit-learn
!pip install bert-score
!pip install evaluate sacremoses sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [9]:
# =========================
# FAISS 기반 문장 리라이팅 시스템
# =========================

# 1. 필요한 라이브러리 및 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')  # Google Drive를 마운트하여 데이터셋에 접근

from sentence_transformers import SentenceTransformer  # 문장을 임베딩하기 위한 모델
import faiss  # 빠른 유사도 검색을 위한 라이브러리 (Facebook AI Similarity Search)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 2. 사전 학습된 한국어 문장 임베딩 모델 로드
# - 다양한 태스크(multi-task)에 대해 학습된 Ko-SRoBERTa 기반 모델 사용
# - 문장을 고차원 벡터로 변환하여 유사도 계산 가능하게 함
model = SentenceTransformer("jhgan/ko-sroberta-multitask")

# 3. 데이터셋 로드
# - original_text: 원래 복잡한 문장
# - simple_text: 쉽게 리라이팅된 문장
df = pd.read_csv('/content/drive/MyDrive/combined_dataset.csv')
originals = df["original_text"].tolist()
simples = df["simple_text"].tolist()

# 4. 원문(original_text)을 벡터로 임베딩
# - 각 문장을 고차원 공간의 점(벡터)으로 표현
# - FAISS는 이 벡터들을 기반으로 최근접 이웃을 빠르게 탐색할 수 있음
original_embeddings = model.encode(originals, convert_to_numpy=True, show_progress_bar=True)

# 5. FAISS 인덱스 생성
# - IndexFlatIP: 내적 기반 유사도 검색 (cosine similarity와 유사하게 작동)
# - normalize_L2로 벡터 정규화하면 cosine similarity와 동일한 효과
index = faiss.IndexFlatIP(original_embeddings.shape[1])
faiss.normalize_L2(original_embeddings)  # 벡터를 단위 벡터로 정규화 (L2 norm = 1)
index.add(original_embeddings)  # 인덱스에 모든 원문 벡터 추가 (검색 대상)

# 6. 유사한 문장을 찾는 함수 정의
# - 사용자가 입력한 문장을 임베딩하고, 기존 문장 중 가장 유사한 것을 찾아서 반환
def find_nearest(original_input, top_k=1):
    # 입력 문장을 임베딩하고 정규화
    embedding = model.encode([original_input], convert_to_numpy=True)
    faiss.normalize_L2(embedding)  # cosine similarity를 위해 정규화

    # FAISS를 사용하여 가장 유사한 top-k 문장 검색
    distances, indices = index.search(embedding, top_k)

    # 해당 인덱스를 통해 쉬운 문장(simple_text) 반환
    nearest_simple = [simples[i] for i in indices[0]]
    return nearest_simple[0]  # top-1 결과만 반환


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [3]:
# Retrieve top-1 nearest sentence using FAISS

def find_nearest(original_input, top_k=1):
    # 입력 임베딩
    embedding = model.encode([original_input], convert_to_numpy=True)
    faiss.normalize_L2(embedding)

    # 최근접 검색
    distances, indices = index.search(embedding, top_k)

    # 결과 반환
    nearest_simple = [simples[i] for i in indices[0]]
    return nearest_simple[0]


In [6]:
# Sample 10% of the dataset for evaluation
df_sampled = df.sample(frac=0.1, random_state=42).reset_index(drop=True)

# Prepare input and reference lists
originals_sample = df_sampled["original_text"].tolist()
references_sample = df_sampled["simple_text"].tolist()

# Generate rewrite outputs
rewrites = [find_nearest(sentence) for sentence in originals_sample]


In [8]:
import sys
sys.path.append('/content/drive/MyDrive')

from evaluate_utils import evaluate_model

evaluate_model(
    originals=originals_sample,
    rewrites=rewrites,
    references=references_sample,
    model_name="Ko-SRoBERTa + FAISS Retrieval"
)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

평가 결과 - Ko-SRoBERTa Retrieval
──────────────────────────────
KoBERTScore:     0.7690
SARI Score:      98.94
평균 문장 길이:   10.53 단어
──────────────────────────────
[1] Original : 맨손체조, 가벼운 산책이나 1주일에 2-3회, 30분 정도의 걷기운동은 가능합니다.
[1] Rewrite  : 체조나 산책처럼 쉬운 운동은 일주일에 2~3번, 한 번에 30분 정도 해도 괜찮아요.
----------------------------------------
[2] Original : 복대는 수술 부위를 보호하고 지지하는 효과가 있으므로 1개월 정도까지 산책하거나 활동할 때 착용하십시오.
[2] Rewrite  : 배에 차는 복대는 아픈 데를 덜 아프게 해줘요. 걷거나 움직일 때는 한 달쯤 착용하세요.
----------------------------------------
[3] Original : 직장에 복귀한 후에는 서서히 활동 시간을 늘려 나가고, 업무량을 조절하여 처음부터 무리하지 않도록 합니다.
[3] Rewrite  : 회사에 다시 나가면 일을 너무 많이 하지 말고 천천히 늘려가요.
----------------------------------------
[4] Original : 이 기간에는 수술로 인한 체력과 상처회복을 위해단백질(고기, 생선, 콩/두부 등)과 비타민 C(과일, 익힌 야채)를 많이 드십시오.
[4] Rewrite  : 몸을 낫게 하려면 고기나 콩, 과일 같은 걸 많이 먹어요.
----------------------------------------
[5] Original : 수술 후 장관이 부분적으로 또는 완전히 막혀 음식물, 소화액, 가스 등 장 내용물이 통과하지 못하는 상태를 말합니다.
[5] Rewrite  : 배 속이 막혀서 음식이나 공기가 안 나가는 경우가 생길 수 있어요.
---------------